# Notebook 3: Neural networks with Keras

On to the most exciting machine learning technique: Neural networks. as you will see they are just as easy to use as the sklearn methods.

In [1]:
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
# Load the data
with open('./data/preproc_data.pkl', 'rb') as f:
    X_train, y_train, X_valid, y_valid, X_test = pickle.load(f)
with open('./data/df_train.pkl', 'rb') as f:
    df_train = pickle.load(f)

## Linear regression: The Keras way

In [3]:
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


In [4]:
X_train.shape

(728008, 22)

In [33]:
ln = Sequential([Dense(1, input_shape=(22,), activation='linear')])

In [34]:
ln.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1)                 23        
Total params: 23
Trainable params: 23
Non-trainable params: 0
_________________________________________________________________


In [35]:
ln.compile(Adam(1e-1), 'mse')

In [36]:
ln.fit(X_train, y_train, 10_000, epochs=12, validation_data=(X_valid, y_valid))

Train on 728008 samples, validate on 180849 samples
Epoch 1/12
728008/728008 [==============================] - 1s 1us/step - loss: 27.4319 - val_loss: 6.8245
Epoch 2/12
728008/728008 [==============================] - 1s 1us/step - loss: 3.8454 - val_loss: 3.5604
Epoch 3/12
728008/728008 [==============================] - 1s 1us/step - loss: 3.1538 - val_loss: 3.4450
Epoch 4/12
728008/728008 [==============================] - 1s 1us/step - loss: 3.0922 - val_loss: 3.3622
Epoch 5/12
728008/728008 [==============================] - 1s 1us/step - loss: 3.0536 - val_loss: 3.3181
Epoch 6/12
728008/728008 [==============================] - 1s 1us/step - loss: 3.0309 - val_loss: 3.2888
Epoch 7/12
728008/728008 [==============================] - 1s 1us/step - loss: 3.0183 - val_loss: 3.2647
Epoch 8/12
728008/728008 [==============================] - 1s 1us/step - loss: 3.0108 - val_loss: 3.2571
Epoch 9/12
728008/728008 [==============================] - 1s 1us/step - loss: 3.0076 - val_loss: 

In [39]:
ln.evaluate(X_valid, y_valid, 10000)

180849/180849 [==============================] - 0s 1us/step


3.2519269126155113

Recall that with the sklearn linear regression algorithm we got a score of around 3.24. This indicates that we are doing pretty much the same here. But we can do better!

## Neural network with a hidden layer

In [52]:
nn = Sequential([
    Dense(256, input_shape=(22,), activation='relu'),
    Dense(1, activation='linear')
])

In [53]:
nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 256)               5888      
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 257       
Total params: 6,145
Trainable params: 6,145
Non-trainable params: 0
_________________________________________________________________


In [41]:
nn.compile(Adam(1e-3), 'mse')

In [43]:
nn.fit(X_train, y_train, 1024, epochs=12, validation_data=(X_valid, y_valid))

Train on 728008 samples, validate on 180849 samples
Epoch 1/12
728008/728008 [==============================] - 4s 6us/step - loss: 9.0466 - val_loss: 3.3911
Epoch 2/12
728008/728008 [==============================] - 4s 5us/step - loss: 2.8325 - val_loss: 3.0486
Epoch 3/12
728008/728008 [==============================] - 4s 6us/step - loss: 2.6490 - val_loss: 2.9720
Epoch 4/12
728008/728008 [==============================] - 4s 5us/step - loss: 2.5792 - val_loss: 2.9627
Epoch 5/12
728008/728008 [==============================] - 4s 5us/step - loss: 2.5340 - val_loss: 2.9400
Epoch 6/12
728008/728008 [==============================] - 4s 5us/step - loss: 2.4965 - val_loss: 2.9030
Epoch 7/12
728008/728008 [==============================] - 4s 6us/step - loss: 2.4643 - val_loss: 2.9304
Epoch 8/12
728008/728008 [==============================] - 4s 6us/step - loss: 2.4347 - val_loss: 2.9048
Epoch 9/12
728008/728008 [==============================] - 4s 6us/step - loss: 2.4104 - val_loss: 2

## ADVANCED TECHNIQUE ALERT: Embeddings

In [58]:
split_date = '2015-01-01'

In [59]:
stations_train = df_train.station[df_train.time < split_date]
stations_valid = df_train.station[df_train.time >= split_date]

In [67]:
stations = df_train.station

In [68]:
unique_stations = stations.unique()

In [69]:
stat2id = {s: i for i, s in enumerate(unique_stations)}

In [70]:
ids = stations.apply(lambda x: stat2id[x])

In [72]:
ids_train = ids[df_train.time < split_date]
ids_valid = ids[df_train.time >= split_date]

In [73]:
from keras.models import Model

In [95]:
features_in = Input(shape=(22,))
id_in = Input(shape=(1,))
emb = Embedding(len(unique_stations), 2)(id_in)
emb = Flatten()(emb)
x = Concatenate()([features_in, emb])
x = Dense(100, activation='relu')(x)
#x = Dropout(0.2)(x)
out = Dense(1, activation='linear')(x)
model = Model(inputs=[features_in, id_in], outputs=out)

In [96]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1, 2)         1040        input_12[0][0]                   
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 22)           0                                            
__________________________________________________________________________________________________
flatten_6 (Flatten)             (None, 2)            0           embedding_6[0][0]                
__________________________________________________________________________________________________
concatenat

In [97]:
model.compile('adam', 'mse')

In [98]:
model.fit([X_train, ids_train], y_train, 1024, 10, 
          validation_data=([X_valid, ids_valid], y_valid))

Train on 728008 samples, validate on 180849 samples
Epoch 1/10
728008/728008 [==============================] - 5s 7us/step - loss: 10.0533 - val_loss: 3.2571
Epoch 2/10
728008/728008 [==============================] - 5s 6us/step - loss: 2.7089 - val_loss: 2.8566
Epoch 3/10
728008/728008 [==============================] - 5s 6us/step - loss: 2.4836 - val_loss: 2.7187
Epoch 4/10
728008/728008 [==============================] - 5s 6us/step - loss: 2.3735 - val_loss: 2.6649
Epoch 5/10
728008/728008 [==============================] - 5s 7us/step - loss: 2.2991 - val_loss: 2.6173
Epoch 6/10
728008/728008 [==============================] - 5s 6us/step - loss: 2.2401 - val_loss: 2.5528
Epoch 7/10
728008/728008 [==============================] - 5s 7us/step - loss: 2.1935 - val_loss: 2.5359
Epoch 8/10
728008/728008 [==============================] - 5s 7us/step - loss: 2.1600 - val_loss: 2.5222
Epoch 9/10
728008/728008 [==============================] - 5s 6us/step - loss: 2.1313 - val_loss: 

In [99]:
model.fit([X_train, ids_train], y_train, 1024, 10, 
          validation_data=([X_valid, ids_valid], y_valid))

Train on 728008 samples, validate on 180849 samples
Epoch 1/10
728008/728008 [==============================] - 5s 6us/step - loss: 2.0705 - val_loss: 2.4833
Epoch 2/10
728008/728008 [==============================] - 5s 6us/step - loss: 2.0423 - val_loss: 2.4512
Epoch 3/10
728008/728008 [==============================] - 5s 6us/step - loss: 2.0239 - val_loss: 2.4660
Epoch 4/10
728008/728008 [==============================] - 5s 6us/step - loss: 2.0111 - val_loss: 2.4331
Epoch 5/10
728008/728008 [==============================] - 5s 6us/step - loss: 2.0007 - val_loss: 2.4228
Epoch 6/10
728008/728008 [==============================] - 5s 6us/step - loss: 1.9915 - val_loss: 2.4276
Epoch 7/10
728008/728008 [==============================] - 5s 6us/step - loss: 1.9836 - val_loss: 2.4551
Epoch 8/10
728008/728008 [==============================] - 5s 6us/step - loss: 1.9767 - val_loss: 2.4499
Epoch 9/10
728008/728008 [==============================] - 5s 6us/step - loss: 1.9697 - val_loss: 2